# A2A Protocol Quick Start - Interactive Testing

This notebook helps you test the A2A (Agent-to-Agent) Protocol implementation.

**Default**: Tests against `https://colinmcnamara.com` (working implementation)  
**Custom**: Change `SITE_URL` to test your own site

## Step 1: Install Dependencies

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install -q requests tabulate

## Step 2: Import Libraries

In [ ]:
import requests
import json
import time
from datetime import datetime
from typing import Dict, Any, Optional, List
from tabulate import tabulate
from IPython.display import display, Markdown
import os

## Step 3: Create A2A Test Client

In [ ]:
class A2AClient:
    """Simple A2A Protocol client for testing - supports both single and multi-endpoint setups"""
    
    def __init__(self, base_url: str):
        self.base_url = base_url.rstrip('/')
        self.agent_card_url = f"{self.base_url}/.well-known/agent.json"
        self.service_url = f"{self.base_url}/api/a2a/service"
        self.request_id = 0
        self.service_endpoints = None  # Will be populated from agent card
        
    def get_agent_card(self) -> Dict[str, Any]:
        """Fetch the agent card"""
        try:
            response = requests.get(self.agent_card_url, timeout=10)
            response.raise_for_status()
            card = response.json()
            
            # Extract service endpoints if available
            if "serviceEndpoints" in card:
                self.service_endpoints = card["serviceEndpoints"]
                print("✅ Multi-endpoint setup detected")
            else:
                print("ℹ️  Single endpoint setup detected")
                
            return card
        except Exception as e:
            return {"error": str(e)}
    
    def call_method(self, method: str, params: Optional[Dict] = None) -> Dict[str, Any]:
        """Call an A2A method - supports both single and multi-endpoint setups"""
        self.request_id += 1
        
        # Determine endpoint URL
        endpoint_url = self.service_url  # Default to single endpoint
        
        # Check if we have multi-endpoint setup
        if self.service_endpoints and method in self.service_endpoints:
            endpoint_url = f"{self.base_url}{self.service_endpoints[method]}"
            print(f"→ Using endpoint: {endpoint_url}")
        
        payload = {
            "jsonrpc": "2.0",
            "id": self.request_id
        }
        
        # For multi-endpoint setup, we don't include the method in the payload
        if not self.service_endpoints or method not in self.service_endpoints:
            payload["method"] = method
            
        if params:
            payload["params"] = params
            
        try:
            response = requests.post(
                endpoint_url,
                json=payload,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
            return response.json()
        except Exception as e:
            return {"error": str(e)}

print("✅ A2A Client created with multi-endpoint support!")

## Step 4: Configure Test Site

In [ ]:
# Configure your site URL
SITE_URL = "https://colinmcnamara.com"  # Default: working A2A implementation

# To test your own site, uncomment and modify:
# SITE_URL = "https://yourdomain.com"
# SITE_URL = "http://localhost:4321"

# Create client
client = A2AClient(SITE_URL)
print(f"Testing A2A at: {SITE_URL}")

## Step 5: Test Agent Card Discovery

In [ ]:
print("🔍 Testing agent card discovery...\n")
agent_card = client.get_agent_card()

if "error" not in agent_card:
    print("✅ Agent Card Found!\n")
    
    # Extract capabilities/skills
    capabilities = []
    if "capabilities" in agent_card:
        capabilities = agent_card["capabilities"]
    elif "skills" in agent_card:
        capabilities = [skill.get("method", skill.get("id", "?")) for skill in agent_card["skills"]]
    
    # Display info
    info = [
        ["Name", agent_card.get("name", "N/A")],
        ["Version", agent_card.get("version", "N/A")],
        ["Methods", len(capabilities)],
        ["Protocol", agent_card.get("protocol", "N/A")],
    ]
    print(tabulate(info, headers=["Property", "Value"], tablefmt="grid"))
    
    if capabilities:
        print("\nAvailable methods:")
        for cap in capabilities:
            print(f"  • {cap}")
            
    # Show service endpoints if available
    if "serviceEndpoints" in agent_card:
        print("\nService Endpoints (Multi-endpoint setup):")
        for method, endpoint in agent_card["serviceEndpoints"].items():
            print(f"  • {method:<25} → {endpoint}")
    else:
        print(f"\nSingle Service Endpoint: {client.service_url}")
else:
    print(f"❌ Error: {agent_card['error']}")

## Step 6: Test Blog Metadata

In [ ]:
print("\n📊 Testing blog metadata...\n")

# Test the metadata endpoint
response = client.call_method("blog.get_metadata")

if "result" in response:
    metadata = response["result"]
    print("✅ Blog Metadata Retrieved!\n")
    
    # Display blog metadata
    info = [
        ["Name", metadata.get("name", "N/A")],
        ["Description", metadata.get("description", "N/A")[:50] + "..." if metadata.get("description") else "N/A"],
        ["URL", metadata.get("url", "N/A")],
        ["Language", metadata.get("language", "N/A")],
        ["Total Posts", metadata.get("totalPosts", "N/A")],
        ["Total Words", f"{metadata.get('totalWords', 0):,}" if metadata.get('totalWords') else "N/A"],
    ]
    print(tabulate(info, headers=["Property", "Value"], tablefmt="grid"))
    
    # Display author info if included
    if "author" in metadata:
        author = metadata["author"]
        print("\n👤 Author Information:\n")
        auth_info = [
            ["Name", author.get("name", "N/A")],
            ["Email", author.get("email", "N/A")],
            ["Bio", author.get("bio", "N/A")[:50] + "..." if author.get("bio") else "N/A"],
        ]
        print(tabulate(auth_info, headers=["Property", "Value"], tablefmt="grid"))
else:
    print(f"❌ Error: {response.get('error', 'Unknown error')}")
    
# Test the separate author endpoint if available
print("\n👤 Testing author info endpoint...\n")
author_response = client.call_method("blog.get_author_info")

if "result" in author_response:
    author = author_response["result"]
    print("✅ Author Info Retrieved!\n")
    
    auth_info = [
        ["Name", author.get("name", "N/A")],
        ["Email", author.get("email", "N/A")],
        ["Bio", author.get("bio", "N/A")[:80] + "..." if len(author.get("bio", "")) > 80 else author.get("bio", "N/A")],
    ]
    print(tabulate(auth_info, headers=["Property", "Value"], tablefmt="grid"))
    
    if "social" in author:
        print("\n🔗 Social Links:")
        for platform, handle in author["social"].items():
            print(f"  • {platform}: {handle}")
else:
    print(f"ℹ️  Author endpoint not available separately")

## Step 7: Test List Posts

In [ ]:
print("\n📚 Testing list posts...\n")

response = client.call_method("blog.list_posts", {"limit": 3})

if "result" in response:
    result = response["result"]
    posts = result.get("posts", [])
    print(f"✅ Found {len(posts)} posts\n")
    
    if posts:
        for i, post in enumerate(posts, 1):
            print(f"{i}. {post.get('title', 'N/A')}")
            print(f"   Date: {post.get('date', 'N/A')[:10]}")
            print(f"   ID: {post.get('id', 'N/A')}")
            print()
    else:
        print("ℹ️  No posts returned (empty blog or response issue)")
else:
    print(f"❌ Error: {response.get('error', 'Unknown error')}")

## Step 8: Test Search

In [ ]:
print("\n🔍 Testing search...\n")

response = client.call_method("blog.search_posts", {"query": "AI", "limit": 3})

if "result" in response:
    result = response["result"]
    posts = result.get("posts", [])
    print(f"✅ Found {len(posts)} posts matching 'AI'\n")
    
    if posts:
        for post in posts:
            print(f"• {post.get('title', 'N/A')}")
else:
    print(f"❌ Error: {response.get('error', 'Unknown error')}")

## Step 9: Test Additional Methods

In [ ]:
print("\n🧪 Testing all blog methods...\n")

# Test blog.get_post with a real post slug
print("📖 Testing blog.get_post...")
# First get a post slug from list_posts
list_response = client.call_method("blog.list_posts", {"limit": 1})
if "result" in list_response and "posts" in list_response["result"] and list_response["result"]["posts"]:
    first_post = list_response["result"]["posts"][0]
    post_slug = first_post.get("slug", first_post.get("id"))
    
    # Now fetch the full post
    post_response = client.call_method("blog.get_post", {"slug": post_slug})
    if "result" in post_response:
        post = post_response["result"]
        print(f"✅ Successfully fetched post: {post.get('title', 'N/A')}")
        print(f"   - Content length: {len(post.get('content', ''))} characters")
        print(f"   - Has metadata: {'_a2a' in post}")
    else:
        print(f"❌ Error fetching post: {post_response.get('error', 'Unknown')}")
else:
    print("❌ Could not get a post slug to test with")

# Test search with tags
print("\n🏷️  Testing search with tags...")
tag_search = client.call_method("blog.search_posts", {"tags": ["AI"], "limit": 2})
if "result" in tag_search and "posts" in tag_search["result"]:
    posts = tag_search["result"]["posts"]
    print(f"✅ Found {len(posts)} posts with tag 'AI'")
    for post in posts:
        print(f"   • {post.get('title', 'N/A')}")
else:
    print("❌ Tag search not working or no posts with AI tag")

# Test pagination
print("\n📄 Testing pagination...")
page1 = client.call_method("blog.list_posts", {"limit": 2, "offset": 0})
page2 = client.call_method("blog.list_posts", {"limit": 2, "offset": 2})

if "result" in page1 and "result" in page2:
    if "posts" in page1["result"] and "posts" in page2["result"]:
        p1_posts = page1["result"]["posts"]
        p2_posts = page2["result"]["posts"]
        
        if p1_posts and p2_posts and p1_posts[0]["id"] != p2_posts[0]["id"]:
            print("✅ Pagination works correctly")
            print(f"   - Page 1: {p1_posts[0]['title'][:40]}...")
            print(f"   - Page 2: {p2_posts[0]['title'][:40]}...")
        else:
            print("⚠️  Pagination might not be working correctly")
    else:
        print("❌ Response missing posts array")
else:
    print("❌ Pagination test failed")

# Show endpoint usage summary
if client.service_endpoints:
    print("\n📊 Endpoint Usage Summary:")
    print("This site uses individual endpoints for each method:")
    methods_tested = ["blog.list_posts", "blog.get_post", "blog.search_posts", 
                      "blog.get_metadata", "blog.get_author_info"]
    for method in methods_tested:
        if method in client.service_endpoints:
            print(f"   ✅ {method} → {client.service_endpoints[method]}")
        else:
            print(f"   ❌ {method} (not mapped)")

## Step 10: Performance Test

In [ ]:
print("\n⚡ Running performance test...\n")

methods = [
    ("blog.get_metadata", None),
    ("blog.list_posts", {"limit": 10}),
    ("blog.get_author_info", None),
]

for method, params in methods:
    times = []
    
    for _ in range(3):
        start = time.time()
        client.call_method(method, params)
        elapsed = (time.time() - start) * 1000
        times.append(elapsed)
        time.sleep(0.1)
    
    avg_time = sum(times) / len(times)
    print(f"{method}: {avg_time:.1f}ms avg")

## Step 11: Generate Report

In [ ]:
print("\n" + "="*50)
print("A2A IMPLEMENTATION REPORT")
print("="*50)
print(f"Site: {SITE_URL}")
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print()

# Test all features
features = {
    "Agent Card": "error" not in client.get_agent_card(),
    "Blog Metadata": "result" in client.call_method("blog.get_metadata"),
    "Author Info": "result" in client.call_method("blog.get_author_info"),
    "List Posts": False,  # Will test below
    "Get Post": False,  # Will test below
    "Search": False,  # Will test below
    "Multi-endpoint Setup": client.service_endpoints is not None,
}

# Test list posts
list_resp = client.call_method("blog.list_posts", {"limit": 1})
if "result" in list_resp and "posts" in list_resp["result"]:
    features["List Posts"] = True
    
    # Test get_post with a real slug if we have posts
    if list_resp["result"]["posts"]:
        slug = list_resp["result"]["posts"][0].get("slug", list_resp["result"]["posts"][0].get("id"))
        if slug:
            features["Get Post"] = "result" in client.call_method("blog.get_post", {"slug": slug})

# Test search
search_resp = client.call_method("blog.search_posts", {"query": "test"})
if "result" in search_resp and "posts" in search_resp["result"]:
    features["Search"] = True

print("Feature Status:")
for feature, status in features.items():
    print(f"  {'✅' if status else '❌'} {feature}")

# Check endpoint configuration
if client.service_endpoints:
    print(f"\n🔌 Multi-endpoint Configuration:")
    print(f"  Total endpoints defined: {len(client.service_endpoints)}")
    
    # Expected endpoints for our implementation
    expected_endpoints = {
        "blog.list_posts": "/api/a2a/blog/list",
        "blog.get_post": "/api/a2a/blog/get",
        "blog.search_posts": "/api/a2a/blog/search",
        "blog.get_metadata": "/api/a2a/blog/metadata",
        "blog.get_author_info": "/api/a2a/blog/author"
    }
    
    missing_endpoints = []
    for method, endpoint in expected_endpoints.items():
        if method not in client.service_endpoints:
            missing_endpoints.append(method)
        elif client.service_endpoints[method] != endpoint:
            print(f"  ⚠️  {method} has unexpected endpoint: {client.service_endpoints[method]}")
    
    if missing_endpoints:
        print(f"  ❌ Missing endpoints: {', '.join(missing_endpoints)}")
    else:
        print("  ✅ All expected endpoints are properly mapped")
else:
    print("\n⚠️  Single endpoint configuration (legacy)")

# Calculate score
score = sum(features.values()) / len(features) * 100
print(f"\nImplementation Score: {score:.0f}%")

if score == 100:
    print("\n🎉 Your A2A implementation is complete and using the latest multi-endpoint architecture!")
elif score >= 85:
    print("\n👍 Excellent! Your implementation is nearly complete.")
elif score >= 70:
    print("\n👍 Good progress! Most features are working.")
else:
    print("\n🚧 Implementation needs work. Check the missing features above.")

# Additional recommendations
print("\n📋 Recommendations:")
if not client.service_endpoints:
    print("  • Consider upgrading to multi-endpoint setup for better performance")
    print("  • Add serviceEndpoints mapping to your agent card")
    
if not features["Get Post"]:
    print("  • Ensure blog.get_post endpoint accepts 'slug' parameter")
    
if not features["Search"]:
    print("  • Implement search functionality with query and tag support")

print("\n🔗 Useful Links:")
print("  • A2A Protocol Spec: https://github.com/colinmcnamara/agent-to-agent-protocol")
print("  • Implementation Guide: ./a2a-implementation-guide.md")
print("  • Testing Guide: ./a2a-testing-guide.md")

## Next Steps

1. If using your own site and tests fail:
   - Ensure agent card exists at `/.well-known/agent.json`
   - Verify API endpoint at `/api/a2a/service` or individual endpoints
   - Check CORS headers are set

2. For full implementation details:
   - See [A2A Implementation Guide](./a2a-implementation-guide.md)
   - Check [A2A Testing Guide](./a2a-testing-guide.md)

3. Test with AI agents:
   - Try at [Google AI Studio](https://aistudio.google.com)
   - Add your agent card URL